In [8]:
import numpy as np
import scipy as sp
import matplotlib.pylab as plt
import math
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
import time
from sklearn.metrics.pairwise import rbf_kernel
%matplotlib inline

def createK(data, c):
    return rbf_kernel(data, data, gamma=1/c)

def calcBetaK(alphaK, data, x, c):
    kernel = rbf_kernel(data, x.reshape(1, -1), 1/c)
    return np.dot(alphaK, kernel)

def centerK(K):
    ''' Returns centered K matrix, see K. Murphy 14.43 '''
    l = len(K)
    l_ones = np.ones((l, l), dtype=int) / l
    Kcentered = K - np.dot(l_ones,K)-np.dot(K,l_ones)+np.dot(l_ones,np.dot(K,l_ones))
    return Kcentered

def normAlpha(alpha, lambdas):
    ''' Returns new alpha corresponding to normalized eigen vectors,
        so that lambda_k(a^k * a^k) = 1 '''
    for i,a in enumerate(alpha):
        a /= np.sqrt(lambdas[i])
    return alpha

def calcZ(alpha, data, x, c, z0, maxIters, showProgress=False):
    ''' Equation (10), returns pre-image z for single input datapoint x '''
    z = z0
    cov_0 = np.std(data, axis=0)**2*np.identity(z0.size)
    m = np.mean(data, axis=0)
    iters=0
    beta = [calcBetaK(aK, data, x, c) for aK in alpha]
    gamma = [calcGammaIOpt(alpha, i, beta) for i in range(len(data))]
    
    while iters < maxIters:
        numerator, denom = 0, 0
        kernel = rbf_kernel(data, z.reshape(1, -1), 1/c)
        for i, xi in enumerate(data):
            gammaI = gamma[i] * kernel[i][0]
            numerator += gammaI * xi
            denom += gammaI
        if denom!=0: #handling numerical instability
            z = numerator/denom
            if showProgress:
                print(iters, z)
            iters +=1
        else:
            # print("instability", "z", z, "x", x, "numerator", numerator)
            iters =0
            z = z0 + np.random.multivariate_normal(np.zeros(z0.size), cov_0)
            numerator, denom = 0, 0
    return z

def calcGammaIOpt(alpha, i, beta):
    ''' returns gamma_i = sum_{k=1}^n beta_k * alpha_i^k '''
    gammaI = 0
    alphaI = alpha.T[i]
    for k, alphaKI in enumerate(alphaI):
        gammaI += beta[k] * alphaKI
    return gammaI

def myPCA(data, test, n):
    l, d = data.shape
    C = np.zeros((d, d))
    m = np.mean(data, axis=0)
    for xi in data-m:
        C += 1/l*np.dot(xi.reshape((d, 1)), xi.reshape((d, 1)).T)
    lambdas, v = np.linalg.eigh(C)
    v = v.T
    Z = np.zeros(test.shape) # result
    for i, xi in enumerate(test-m):
        Z[i] = m+np.sum(np.array([np.dot(np.dot(xi, vi), vi) for vi in v[-n:]]), axis=0)
    return Z

def myKPCA(data, test, n, c, maxIters, showProgress=False, debug=False):
    l = len(data)
    K = centerK(createK(data, c))
    lambdas, alphas = np.linalg.eigh(K)
    alphas = alphas.T
    lambdas = lambdas/l
    lambdas, alphas = lambdas[-n:], alphas[-n:]
    alphas = normAlpha(alphas, lambdas)
    Z = np.zeros(test.shape)
    for i, xi in enumerate(test):
        Z[i] = calcZ(alphas, data, xi, c, xi, maxIters, showProgress=showProgress)
    return Z

In [5]:
sigmas = np.array([0.05, 0.1, 0.2, 0.4, 0.8])
centers = np.random.uniform(-1, 1, size=(11, 10))
nTrain, nTest = 100, 33

Xtrain, Xtest = [], []
for s in sigmas:
    xtrain, xtest = [], []
    for center in centers:
        for _ in range(nTrain):
            xtrain.append(np.random.multivariate_normal(center, s**2*np.identity(10)))
        for _ in range(nTest):
            xtest.append(np.random.multivariate_normal(center, s**2*np.identity(10)))
    Xtrain.append(xtrain)
    Xtest.append(xtest)
Xtrain, Xtest = np.array(Xtrain), np.array(Xtest)
Z0 = []
for center in centers:
    for _ in range(nTest):
        Z0.append(center)
Z0 = np.array(Z0)
print(Xtrain.shape, Xtest.shape)

(5, 1100, 10) (5, 363, 10)


In [9]:
start_time = time.time()
maxIters = 1
for sigmaIdx in [0]:
    c = 10*2*sigmas[sigmaIdx]**2
    Data = Xtrain[sigmaIdx]
    Test = Xtest[sigmaIdx]
    for n in [1]:
        Zpca = myPCA(Data, Test, n)
        Zkpca = myKPCA(Data, Test, n, c, maxIters)
        
        errorPCA, errorKPCA = 0, 0
        
        for i, z in enumerate(Zpca):
            errorPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        for i, z in enumerate(Zkpca):
            errorKPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        
        print(sigmas[sigmaIdx], n, errorPCA, errorKPCA, errorPCA/errorKPCA)
print("--- ", time.time()-start_time, "sec", "---")

0.05 1 715.978257009 0.150130456085 4769.04071085
---  12.282093048095703 sec ---


In [147]:
start_time = time.time()
maxIters = 5
for sigmaIdx in [1]:
    c = 10*2*sigmas[sigmaIdx]**2
    Data = Xtrain[sigmaIdx]
    Test = Xtest[sigmaIdx]
    for n in range(1, 10):
        Zpca = myPCA(Data, Test, n)
        Zkpca = myKPCA(Data, Test, n, c, maxIters)
        
        errorPCA, errorKPCA = 0, 0
        
        for i, z in enumerate(Zpca):
            errorPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        for i, z in enumerate(Zkpca):
            errorKPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        
        print(sigmas[sigmaIdx], n, errorPCA, errorKPCA, errorPCA/errorKPCA)
print("--- ", time.time()-start_time, "sec", "---")

0.1 1 914.834142941 0.398880599644 2293.50372958
0.1 2 657.742815909 0.399094877294 1648.08634069
0.1 3 439.928859003 0.399534612097 1101.10324784
0.1 4 298.085559971 0.399708297801 745.757747865
0.1 5 182.667701498 0.399826791118 456.867087338
0.1 6 119.191621049 0.400148079545 297.868781938
0.1 7 66.9153887701 0.400606436563 167.035231246
0.1 8 36.2474835593 0.400863539332 90.4234982799
0.1 9 31.3890052837 0.400961371002 78.2843624194
---  364.1133449077606 sec ---


In [148]:
start_time = time.time()
maxIters = 10
for sigmaIdx in [2]:
    c = 10*2*sigmas[sigmaIdx]**2
    Data = Xtrain[sigmaIdx]
    Test = Xtest[sigmaIdx]
    for n in range(1, 10):
        Zpca = myPCA(Data, Test, n)
        Zkpca = myKPCA(Data, Test, n, c, maxIters)
        
        errorPCA, errorKPCA = 0, 0
        
        for i, z in enumerate(Zpca):
            errorPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        for i, z in enumerate(Zkpca):
            errorKPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        
        print(sigmas[sigmaIdx], n, errorPCA, errorKPCA, errorPCA/errorKPCA)
print("--- ", time.time()-start_time, "sec", "---")

0.2 1 928.732061107 124.814388982 7.44090540105
0.2 2 678.891557631 123.884548536 5.48003415803
0.2 3 473.38301713 20.3941951701 23.2116547469
0.2 4 341.580557353 40.4394840354 8.44670908892
0.2 5 236.111345022 10.4165343941 22.6669769511
0.2 6 181.818938029 9.56128772769 19.0161558995
0.2 7 140.265712988 9.37235363299 14.9659006137
0.2 8 121.620347135 4.23365978862 28.727000564
0.2 9 128.053372872 1.94612721157 65.799076294
---  609.0764348506927 sec ---


In [149]:
start_time = time.time()
maxIters = 10
for sigmaIdx in [3]:
    c = 10*2*sigmas[sigmaIdx]**2
    Data = Xtrain[sigmaIdx]
    Test = Xtest[sigmaIdx]
    for n in range(1, 10):
        Zpca = myPCA(Data, Test, n)
        Zkpca = myKPCA(Data, Test, n, c, maxIters)
        
        errorPCA, errorKPCA = 0, 0
        
        for i, z in enumerate(Zpca):
            errorPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        for i, z in enumerate(Zkpca):
            errorKPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        
        print(sigmas[sigmaIdx], n, errorPCA, errorKPCA, errorPCA/errorKPCA)
print("--- ", time.time()-start_time, "sec", "---")

0.4 1 969.629920331 1394.13736511 0.695505295676
0.4 2 769.414606356 922.251798604 0.834278238894
0.4 3 607.184684833 400.752506216 1.51511138524
0.4 4 520.198217335 264.919557937 1.96360820389
0.4 5 453.62100609 146.116125155 3.10452392306
0.4 6 441.362933958 123.838328507 3.56402528425
0.4 7 446.075653637 91.7325035597 4.8627873036
0.4 8 474.150754377 65.6953067793 7.2174220294
0.4 9 517.487107352 50.1599167846 10.316745731
---  645.8423719406128 sec ---


In [150]:
start_time = time.time()
maxIters = 10
for sigmaIdx in [4]:
    c = 2*sigmas[sigmaIdx]**2
    Data = Xtrain[sigmaIdx]
    Test = Xtest[sigmaIdx]
    for n in range(1, 10):
        Zpca = myPCA(Data, Test, n)
        Zkpca = myKPCA(Data, Test, n, c, maxIters)
        
        errorPCA, errorKPCA = 0, 0
        
        for i, z in enumerate(Zpca):
            errorPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        for i, z in enumerate(Zkpca):
            errorKPCA += np.linalg.norm(z-centers[int(i/nTest)])**2
        
        print(sigmas[sigmaIdx], n, errorPCA, errorKPCA, errorPCA/errorKPCA)
print("--- ", time.time()-start_time, "sec", "---")

0.8 1 1144.91134352 1676.72757622 0.682824902365
0.8 2 1096.93422335 1241.43799456 0.883599686941
0.8 3 1104.40016106 1079.73623397 1.02284254832
0.8 4 1192.50681113 1014.28432867 1.17571254669
0.8 5 1305.45552547 1026.20952296 1.27211402375
0.8 6 1451.01441487 953.414267172 1.52191388867
0.8 7 1662.11684961 928.467358933 1.79017262547
0.8 8 1855.48852231 914.917291878 2.02803962585
0.8 9 2060.53625753 901.562764558 2.28551614878
---  640.5740239620209 sec ---
